# Analysis Ratings & Reviews NIVEA.de

## Notebook 2: Further Data Exploration & First Plots

### Reading the data and importing useful modules

In [32]:
import pandas as pd
import warnings
import seaborn as sns # For plotting
import matplotlib.pyplot as plt # For showing plots
import plotly.express as px  # For creating figures
import re # For working with regular expressions
import os # Provides functions for interacting with the operating system
warnings.filterwarnings('ignore')

reviews = pd.read_csv("reviews.csv", sep=';') # Reading the data

### Further Practice Tasks

#### <font color='red'>From here on, only the filtered data set is used (i.e. reviews that were submitted after 2018)!</font>

In [33]:
reviews['date'] = pd.to_datetime(reviews['date'], format='%Y-%m-%d') # # Adjusting the date format
reviews_filtered = reviews[reviews['date'] >= "2019-01-01"] # Removing the reviews that were submitted before 01/01/2019 

#### 1. Create a graph showing the daily number of ratings and reviews over time.

In [34]:
reviews_filtered['date'] = reviews_filtered['date'].dt.date

In [35]:
df_for_plot = reviews_filtered.groupby('date').agg({'date': 'first','id': 'count'})
df_for_plot

,date,id
date,,
2019-01-01,2019-01-01,3
2019-01-02,2019-01-02,4
2019-01-03,2019-01-03,3
2019-01-04,2019-01-04,4
2019-01-06,2019-01-06,1
...,...,...
2021-11-23,2021-11-23,150
2021-11-24,2021-11-24,163
2021-11-25,2021-11-25,56


In [36]:
fig = px.line(df_for_plot, x = 'date', y = 'id', 
              labels= {'id':'Number Ratings & Reviews', 'date' : 'Date (Day)'})

fig.show()

#### 2. We  analyzed that there was a peak of more than 800 ratings and reviews on one particular day. Approximately 200 went to one product. What could be a possible reason for this?

The reason for this are certain NIVEA campaigns. With these, product packages are sent out for testing on a large scale (500-1000) and the recipients are invited to write a review. These actions are often carried out in connection with a product launch and that was the case here.

#### 3. Divide the products into three categories: shower products, body lotion and the rest. For these three categories, please plot the cumulative number of ratings and reviews over the days for the year 2021.

In [37]:
# defining the search strings
dusche = {"shower","pflegedusche","dusche","duschöl","duschgel"}
body = {"lotion", "milk", "milch", "mousse","butter"}

In [38]:
reviews_filtered['product_title'] = reviews_filtered.product_title.astype('string') # changing the type of "product_title" into a string
reviews_filtered['product_title']= reviews_filtered.product_title.str.lower() # The lower()-method returns the lowercased string

In [40]:
# Now the product name is used to search for components from the search strings 
# If there is a match, the category is assigned to the new variable 
reviews_filtered['category'] = reviews_filtered['product_title'].apply(lambda x: 
                                                                     "dusche" if any(i in x for i in dusche) 
                                                                       else ("body" if any(i in x for i in body) 
                                                                             else "other"))


In [41]:
reviews_filtered['product_title'].isin(dusche).value_counts()

False    81685
Name: product_title, dtype: int64

In [43]:
reviews_filtered.category.value_counts()

other     66226
dusche     8426
body       7033
Name: category, dtype: int64

In [44]:
# Now the reviews are grouped by date and category and counted via the ID
reviews_cumulated = reviews_filtered.groupby(['date','category']).agg({'id': 'count', 'date' : 'min'})

# The data frame is sorted by index (date),in order to recieve the correct cumulative sum of the counts
reviews_cumulated = reviews_cumulated.sort_index()

# And now the counts are added cumulatively
reviews_cumulated['count_cum'] = reviews_cumulated.groupby(['category'])['id'].cumsum()
reviews_cumulated.reset_index(level=1, inplace=True)

In [45]:
reviews_cumulated

,category,id,date,count_cum
date,,,,
2019-01-01,other,3,2019-01-01,3
2019-01-02,other,4,2019-01-02,7
2019-01-03,other,3,2019-01-03,10
2019-01-04,dusche,1,2019-01-04,1
2019-01-04,other,3,2019-01-04,13
...,...,...,...,...
2021-11-25,other,48,2021-11-25,66194
2021-11-26,body,5,2021-11-26,7028
2021-11-26,other,25,2021-11-26,66219


In [46]:
fig = px.line(reviews_cumulated, x = 'date', y = 'count_cum', color = 'category',
              labels= {'count_cum':'Kummulative Anzahl Ratings & Reviews', 'date' : 'Datum (Tag)',
                      'category': 'Kategorie'})

fig.show()

You can see a staircase-like progression. This is very likely due to the NIVEA campaigns mentioned above.

The disadvantage of a text-based classification is that you have to create a dictionary that is as comprehensive as possible. And what happens with a product such as "shower mousse" or "body mousse"?  Here you have to think about a more extensive comparison. The derivation of the category according to this procedure can be very complex and therefore time-consuming and yet never 100% correct.

#### 4. Examine the distribution of the stars for the "NIVEA Eau de Toilette" in a histogram. How do you assess the result of the histogram?

In [47]:
reviews_edt = reviews_filtered[reviews_filtered['product_title'].str.contains("eau de toilette")]

In [48]:
fig = px.histogram(reviews_edt, x = 'review_score')
fig.show()

The distribution is extremely skewed. The ratings are concentrated around the star 5.